In [3]:
using LinearAlgebra

In [4]:
function generateSPDmatrix(n)
    A = rand(n,n)
    B = Symmetric(A,:U)
    B = B + n*Matrix(I(n))
    return B
end

generateSPDmatrix (generic function with 1 method)

### 1. Creating the function of Gauss_Seidel

In [5]:
function Gauss_Seidel(A,b,x, tol=1.0e-9)
    """
    Gauss-Seidel method for solving [A]{x} = {b}.
    Parameters:
    A: a SPD matrix. 
    b: a vector 
    x: a vector of the initial values of x
    tol: the tolerance level
    
    Output:
    A vector of x when the norm of the difference between xnext and xprevious is below some tolerance level
    
    """
    
    
    n = length(b) 
    # L*x_next = b - U*x_pre
    
    x_pre = zeros(n)
    x_next = copy(x)
    count = 1 # record the number of iterations
    tol_level = Inf # the tolerance level is initialized as infinity
    while tol_level > tol
        
        for i in 1:n
            if i == 1 # the first x_next
                x_next[i] = (b[i]-dot(A[i,i+1:n],x_pre[i+1:n]))/A[i,i]
                
            elseif i == n # the last x_next
                x_next[i] = (b[i]- dot(A[i,1:i-1],x_next[1:i-1]))/A[i,i]
            else
                x_next[i] = (b[i]-dot(A[i,i+1:n],x_pre[i+1:n]) - dot(A[i,1:i-1],x_next[1:i-1]))/A[i,i]
            end
        end
        
        # compute the tolerance level by norm
        tol_level = norm(x_next - x_pre)
        x_pre = x_next
        count += 1
    end
    
    return x_next,count,tol_level
end

Gauss_Seidel (generic function with 2 methods)

### 2. Creating an example with dimension n = 10000 to test the Gauss_Seidel method and compare with \ operator of Julia

In [6]:
A = generateSPDmatrix(10000);

In [7]:
b = rand(10000);

In [8]:
x = zeros(10000);

In [9]:
@timev x_next,n,tol_level = Gauss_Seidel(A,b,x)

  5.644175 seconds (856.65 k allocations: 3.023 GiB, 8.88% gc time)
elapsed time (ns): 5644174807
gc time (ns):      501074632
bytes allocated:   3246031856
pool allocs:       778463
non-pool GC allocs:14572
malloc() calls:    63612
GC pauses:         141
full collections:  1


([-8.585345748817443e-6, -1.592230680955768e-5, 1.0970215547110309e-6, 1.9894131982291825e-5, -1.940181399056645e-5, 2.0467121716844915e-5, -1.4965370645909289e-5, 4.7108731369054786e-5, 7.878193998560077e-5, 4.6054284787622e-6  …  6.135984599177123e-5, 6.947261664217057e-5, 3.30933558290525e-5, 2.0840957704761036e-5, 1.483896443494519e-5, 2.6101512258176435e-5, 1.2292859375632082e-5, 1.8727874909221906e-5, 6.064146882546894e-5, 3.6053695348673764e-5], 3, 0.0)

In [10]:
@timev x_ori = A\b

  8.727896 seconds (3.72 M allocations: 942.816 MiB, 0.76% gc time)
elapsed time (ns): 8727896107
gc time (ns):      66598806
bytes allocated:   988614048
pool allocs:       3723799
non-pool GC allocs:742
malloc() calls:    12
GC pauses:         9
full collections:  1


10000-element Array{Float64,1}:
 -5.587687678257085e-6 
 -1.2889590385505455e-5
  4.0594793549882724e-6
  2.2890409512671396e-5
 -1.6383731753786528e-5
  2.350201666483611e-5 
 -1.1968123423778862e-5
  5.010611520639096e-5 
  8.175265465252787e-5 
  7.59723307166442e-6  
  2.1667997582476235e-5
  1.1632256293576295e-5
  3.809291967271936e-6 
  ⋮                    
  2.2692700290553396e-5
 -1.1764551683652093e-5
  6.113013510451993e-5 
  6.925271419148009e-5 
  3.286933202125526e-5 
  2.0610755357023963e-5
  1.4613854417597048e-5
  2.5874292937244716e-5
  1.2065732536503148e-5
  1.8503771524608556e-5
  6.042009201740195e-5 
  3.5830054558019e-5   

In [11]:
norm(x_ori - x_next)

0.00010764770997315322

In [12]:
println("The iteration number of Gauss_Seidel is : $n")
println("The norm of x_next and x_previous in the last iteration is : $tol_level")

The iteration number of Gauss_Seidel is : 3
The norm of x_next and x_previous in the last iteration is : 0.0


## Conclusion: Gauss_Seidel method take less time than the \ operator of julia. But the Gauss_Seidel will use more memeory than \ operator.  And even though the x_next and x_pre is close enough and the norm is lessen than the tolerance level, the x_next still has some gap with the exact result.